In [ ]:
# Import necessary packages
import gurobipy as gp
from gurobipy import GRB
import numpy as np
import pandas as pd

# Load the data set that contains distances between 
# cardiac arrests and potential AED locations
covered = pd.read_csv('Example2_distanceMatrix.csv', index_col='ID')

# PARAMETERS
# Coverage cutoff limit 
coverage_distance = 175

# Number of AEDs to be placed 
K = 10

# Compute a_ij, which states whether each cardiac arrest is
# within the coverage distance of each potential AED location
covered = (covered <= coverage_distance).astype(int)
n_cases = covered.shape[0]
n_candidates = covered.shape[1]
A = pd.DataFrame.to_numpy(covered)

# Create a model object
m = gp.Model('AED')

# DECISION VARIABLES
x = m.addMVar(n_candidates, vtype = GRB.BINARY, name="x")
y = m.addMVar(n_cases, vtype = GRB.BINARY, name="y")

# CONSTRAINTS
# Coverage constraint
m.addConstr(A@x>=y)

# Number of AEDs constraint
m.addConstr(x.sum()==K)

# OBJECTIVE
m.setObjective(y.sum(), gp.GRB.MAXIMIZE)

# Optimize
m.optimize()

# Print solutions
print(x.X)
print(y.X)
print(m.objVal)
